In [ ]:
import rootutils

rootutils.setup_root("../", indicator=".project-root", pythonpath=True)
import os

os.chdir("../")
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# plt.savefig('x.png', dpi=500, bbox_inches='tight') # 解决图片不清晰，不完整的问题

In [ ]:
from omegaconf import OmegaConf

config = r"logs/train/runs/2024-05-29_22-12-00/.hydra/config.yaml"
ckpt = r"logs/train/runs/2024-05-29_22-12-00/checkpoints/step_090_loss0.001.ckpt"
config = OmegaConf.load(config)
config.data.train_config.data_dir = "./demo"
config.data.val_config.data_dir = "./demo"
config.data.batch_size = 1
config.data.batch_size_val = 1
# ckpt = r"./demo/step_4815_loss0.006.ckpt"

In [ ]:
data_module = hydra.utils.instantiate(config.data)
data_module.setup()
batch_sample = next(iter(data_module.val_dataloader()))
print(batch_sample["images"].shape)
show_video_line(
    batch_sample["images"][0].numpy(),
    ncols=12,
    vmax=0.6,
    cbar=False,
    out_path=None,
    format="png",
    use_rgb=True,
)

In [ ]:
import torch

module = hydra.utils.instantiate(config.model)
module.load_state_dict(torch.load(ckpt, map_location="cpu")["state_dict"])
#
print(module.model.configs)  # 输入前 7帧图像，预测后12帧图像
# mask_true[:, t - self.configs.pre_seq_length] * frames[:, t]
pre_seq_length = module.model.configs.pre_seq_length

In [ ]:
module.model.configs.pre_seq_length = 18  # 使用一张图像来预测未来18张
module.model.configs.aft_seq_length = (
    module.model.configs.total_length - module.model.configs.pre_seq_length
)
batch_sample_cp = deepcopy(batch_sample)
batch_sample_cp["images"][:, module.model.configs.pre_seq_length :, ...] = 0
show_video_line(
    batch_sample_cp["images"][0].numpy(),
    ncols=12,
    vmax=0.6,
    cbar=False,
    out_path=None,
    format="png",
    use_rgb=True,
)
with torch.no_grad():
    img_gen, logs = module.RNN_test_step(batch_sample_cp, "val")
    print(img_gen.shape)  # 缺少第一张图
img_gen = torch.concat([batch_sample_cp["images"][:, None, 0, ...], img_gen], dim=1)
show_video_line(
    img_gen[0].detach().numpy(),
    ncols=12,
    vmax=0.6,
    cbar=False,
    out_path=None,
    format="png",
    use_rgb=True,
)
# 预测出输入状态越清晰，准确越能代表RNN学到的时间信息越成功

In [ ]:
module.model.configs.pre_seq_length

In [ ]:
from src.utils import show_video_gif_multiple

# 制作对比gif  -> 用于预测的，待预测的真值，预测结果
# pre_seq_length >= 2
show_video_gif_multiple(
    batch_sample["images"][:, : module.model.configs.pre_seq_length, ...][0],
    batch_sample["images"][:, module.model.configs.pre_seq_length :, ...][0],
    img_gen.detach().numpy()[0],
    use_rgb=True,
    out_path=f"demo/pred/pred_pre_seq-{module.model.configs.pre_seq_length}.gif",
)

In [ ]:
# # 制作 gif 单个
# show_video_gif_single(
#     img_gen.detach().numpy()[0],
#     out_path=f"data/demo/pred/pred_pre_seq-{module.model.configs.pre_seq_length}.gif"
# )